# BCR analysis

## Initialization

### Loading packages

In [1]:
#Library the required

suppressMessages(library(Seurat))
suppressMessages(library(dplyr))
suppressMessages(library(cowplot))
suppressMessages(library(ggplot2))

suppressMessages(library(stringr))
suppressMessages(library(Stat2Data))
suppressMessages(library(tidyverse))
suppressMessages(library(patchwork))
# library(pheatmap))
suppressMessages(library(viridis)) 
suppressMessages(library(ggplot2))
suppressMessages(library(ggrepel))
suppressMessages(library(scRepertoire))
suppressMessages(library(circlize))
suppressMessages(library(scales))
suppressMessages(library(scCustomize))
suppressMessages(library(DESeq2)) 
suppressMessages(library(languageserver))

Warning message:
"package 'Seurat' was built under R version 4.3.3"
Warning message:
"package 'SeuratObject' was built under R version 4.3.3"
Warning message:
"package 'sp' was built under R version 4.3.3"


Warning message:
"package 'ggplot2' was built under R version 4.3.3"
Warning message:
"package 'tidyr' was built under R version 4.3.3"
Warning message:
"package 'viridis' was built under R version 4.3.3"
Warning message:
"package 'ggrepel' was built under R version 4.3.3"
Warning message:
"package 'GenomeInfoDb' was built under R version 4.3.3"
Warning message:
"package 'languageserver' was built under R version 4.3.3"


In [21]:
sessionInfo()

R version 4.3.2 (2023-10-31 ucrt)
Platform: x86_64-w64-mingw32/x64 (64-bit)
Running under: Windows 11 x64 (build 22621)

Matrix products: default


locale:
[1] LC_COLLATE=English_United States.utf8 
[2] LC_CTYPE=English_United States.utf8   
[3] LC_MONETARY=English_United States.utf8
[4] LC_NUMERIC=C                          
[5] LC_TIME=English_United States.utf8    

time zone: America/New_York
tzcode source: internal

attached base packages:
[1] stats4    stats     graphics  grDevices utils     datasets  methods  
[8] base     

other attached packages:
 [1] languageserver_0.3.16       DESeq2_1.42.0              
 [3] SummarizedExperiment_1.32.0 Biobase_2.62.0             
 [5] MatrixGenerics_1.14.0       matrixStats_1.2.0          
 [7] GenomicRanges_1.54.1        GenomeInfoDb_1.38.8        
 [9] IRanges_2.36.0              S4Vectors_0.40.2           
[11] BiocGenerics_0.48.1         scCustomize_2.0.1          
[13] scales_1.3.0                circlize_0.4.15            
[15] scRep

### Read in Data

In [22]:
all_samples<- readRDS('merge_all_samples.rds')
glimpse(all_samples)

Rows: 667,716
Columns: 15
$ CDR3         <chr> "AAAAQAWFAY", "AAADYYDSSYDY", "AAAGITTDYAMDY", "AAAGYDPYY…
$ IGH          <chr> "mIGHV1-80", "mIGHV1-9", "mIGHV1-34", "mIGHV5-17", "mIGHV…
$ IGJ          <chr> "mIGHJ3", "mIGHJ2", "mIGHJ4", "mIGHJ2", "mIGHJ2", "mIGHJ3…
$ E1_frequency <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ L1_frequency <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ S1_frequency <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ E2_frequency <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ L2_frequency <int> NA, NA, NA, NA, NA, NA, NA, NA, 2, 3, 1, NA, NA, NA, NA, …
$ S2_frequency <int> NA, 1, 1, NA, NA, 17, NA, NA, NA, NA, NA, 1, NA, 1, 1, NA…
$ E3_frequency <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ L3_frequency <int> NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, NA, N…
$ S3_frequency <int> NA, NA, NA, NA, NA, NA, NA, 1, NA, NA, NA, NA, 2, NA, NA,…
$ E4_frequency

In [23]:
head( all_samples)

,CDR3,IGH,IGJ,E1_frequency,L1_frequency,S1_frequency,E2_frequency,L2_frequency,S2_frequency,E3_frequency,L3_frequency,S3_frequency,E4_frequency,L4_frequency,S4_frequency
,<chr>,<chr>,<chr>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,AAAAQAWFAY,mIGHV1-80,mIGHJ3,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
2,AAADYYDSSYDY,mIGHV1-9,mIGHJ2,NA,NA,NA,NA,NA,1,NA,NA,NA,NA,NA,NA
3,AAAGITTDYAMDY,mIGHV1-34,mIGHJ4,NA,NA,NA,NA,NA,1,NA,NA,NA,NA,NA,NA
4,AAAGYDPYYFDY,mIGHV5-17,mIGHJ2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,1
5,AAAITTAFDY,mIGHV3-8,mIGHJ2,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,2
6,AAALRFAY,mIGHV14-4,mIGHJ3,NA,NA,NA,NA,NA,17,NA,NA,NA,NA,NA,NA


# Remaking file to incorporate SHM and replicates

In [2]:
samples <- c('E1', 'E2', 'E3', 'E4', 'L1', 'L2', 'L3', 'L4', 'S1', 'S2', 'S3', 'S4')

files <- c('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E1__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E1_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E2__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E2_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E3__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E3_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E4__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E4_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L1__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L1_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L2__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L2_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L3__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L3_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L4__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L4_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S1__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S1_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S2__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S2_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S3__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S3_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S4__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S4_r__IGH.csv')

In [3]:
results_all_samples_simple <- tibble()
results_all_samples_with_SHM <- tibble()

for (sample in samples) {
    sample1 <- paste0('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/', sample, '__IGH.csv' )
    sample2 <- paste0('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/', sample, '_r__IGH.csv' )

    table1 <- read.csv(file = sample1)
    table2 <- read.csv(file = sample2)

    table1 <- table1 |>
    select(all_of(c('junction_aa', 'v_call', 'v_mismatches')))

    table2 <- table2 |>
    select(all_of(c('junction_aa', 'v_call', 'v_mismatches')))

    table <- bind_rows(table1, table2)  |>
        rename('junction_aa' = 'CDR3') |>
        mutate(clonotype  =  paste(CDR3, v_call, v_mismatches, sep = '_')) |> 
        select(-c(CDR3, v_call)) |>
        relocate(clonotype) |> 
        add_count(clonotype, v_mismatches, name = sample)  |>
        distinct()  
    
    table_simple  <- bind_rows(table1, table2)  |>
        rename('junction_aa' = 'CDR3') |>
        select(-v_mismatches) |> 
        mutate(clonotype  =  paste(CDR3, v_call,sep = '_')) |> 
        select(-c(CDR3, v_call)) |>
        relocate(clonotype) |> 
        add_count(clonotype, name =  sample)   |>
        distinct()
    
    if (nrow(results_all_samples_simple) == 0) {
        results_all_samples_simple <- table_simple
        results_all_samples_with_SHM <- table 
    } else {
        results_all_samples_simple <- full_join(results_all_samples_simple, table_simple, by = 'clonotype') |>
                mutate_all(~replace(., is.na(.), 0))  

        results_all_samples_with_SHM <- full_join(results_all_samples_with_SHM, table, by = c( 'clonotype' ,  'v_mismatches' )) |>
                mutate_all(~replace(., is.na(.), 0))
    }
        
}

head(results_all_samples_simple)
head(results_all_samples_with_SHM)

write.csv(results_all_samples_simple, file = 'results_all_samples_simple.csv', row.names = F)

write.csv(results_all_samples_with_SHM, file = 'results_all_samples_with_SHM.csv', row.names = F)

,clonotype,E1,E2,E3,E4,L1,L2,L3,L4,S1,S2,S3,S4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CARAPYGNYRFAYW_IGHV9-3,5,0,0,0,0,0,0,0,2,0,0,0
2,CARYRSGYDAMDYW_IGHV1-18,4,0,0,0,0,0,0,0,0,0,0,0
3,CSIYYSFPFEYW_IGHV14-3,8,0,0,0,0,0,0,0,21,0,0,0
4,CARNGYYYAMDYW_IGHV2-2,5,0,0,0,0,0,2,5,7,7,14,2
5,CVRDGDYDVDWFAYW_IGHV10-3,5,0,0,0,0,0,0,0,0,0,0,0
6,CAKRYYGNVGYFDYW_IGHV1-64,4,0,0,0,0,0,0,0,2,0,0,0


,clonotype,v_mismatches,E1,E2,E3,E4,L1,L2,L3,L4,S1,S2,S3,S4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CARAPYGNYRFAYW_IGHV9-3_0,0,3,0,0,0,0,0,0,0,1,0,0,0
2,CARYRSGYDAMDYW_IGHV1-18_0,0,2,0,0,0,0,0,0,0,0,0,0,0
3,CSIYYSFPFEYW_IGHV14-3_11,11,3,0,0,0,0,0,0,0,5,0,0,0
4,CARNGYYYAMDYW_IGHV2-2_5,5,2,0,0,0,0,0,0,0,2,0,3,0
5,CVRDGDYDVDWFAYW_IGHV10-3_4,4,1,0,0,0,0,0,0,0,0,0,0,0
6,CAKRYYGNVGYFDYW_IGHV1-64_1,1,2,0,0,0,0,0,0,0,1,0,0,0


# Percentage overlap final

## Total sequences

In [7]:
# results_all_samples_simple
# results_all_samples_with_SHM

results <- tibble(.rows = 8)
sequences_results <- tibble(.rows = 4)

for (Mouse in seq(1:4)) {
    
    mouse <- as.character(Mouse)
    skin_mouse <- paste0('E',  mouse )
    spleen_mouse <- paste0('S',  mouse )
    LN_mouse <- paste0('L',  mouse )
    
    new_names <- c('skin', 'spleen', 'LN')
    names(new_names) <- c(skin_mouse, spleen_mouse, LN_mouse)

    results_all_samples_simple_skin <- results_all_samples_simple |> 
        select(contains(mouse) | clonotype) |> 
        rename(all_of(new_names)) |> 
        filter(skin > 0)         

    total_unique_clones <- results_all_samples_simple_skin |> pull(clonotype) |> n_distinct()
    total_unique_sequences <- results_all_samples_simple_skin |> pull(skin) |> sum()


    clones_percent_skin_unique <- results_all_samples_simple_skin |> filter(LN == 0 & spleen == 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_skin_unique <- results_all_samples_simple_skin |> filter(LN == 0 & spleen == 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_LN_only <- results_all_samples_simple_skin |> filter(LN  >  0 & spleen == 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_LN_only <- results_all_samples_simple_skin |> filter(LN > 0 & spleen == 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_spleen_only <- results_all_samples_simple_skin |> filter(LN ==  0 & spleen > 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_spleen_only <- results_all_samples_simple_skin |> filter(LN  ==  0 & spleen > 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_spleen_and_LN <- results_all_samples_simple_skin |> filter(LN >  0 & spleen > 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_spleen_and_LN <- results_all_samples_simple_skin |> filter(LN  >  0 & spleen > 0) |> pull(skin) |> sum()*100/total_unique_sequences

    mouse_results <- c(clones_percent_skin_unique, clones_percent_overlap_with_LN_only, clones_percent_overlap_with_spleen_only, clones_percent_overlap_with_spleen_and_LN, sequences_percent_skin_unique, sequences_percent_overlap_with_LN_only, sequences_percent_overlap_with_spleen_only, sequences_percent_overlap_with_spleen_and_LN)


    results <- bind_cols(results, tibble(mouse = mouse_results))

}
row_names  <-  c('clones_percent_skin_unique', 'clones_percent_overlap_with_LN_only', 'clones_percent_overlap_with_spleen_only', 'clones_percent_overlap_with_spleen_and_LN', 'sequences_percent_skin_unique', 'sequences_percent_overlap_with_LN_only', 'sequences_percent_overlap_with_spleen_only', 'sequences_percent_overlap_with_spleen_and_LN')
results <- results |> mutate(Metric = row_names) |> relocate(Metric)
head(results)

write.csv(results, file = 'BCR_total_overlap_between_tissues.csv', row.names = FALSE)





New names:
• `mouse` -> `mouse...1`
• `mouse` -> `mouse...2`
New names:
• `mouse` -> `mouse...3`
New names:
• `mouse` -> `mouse...4`


Metric,mouse...1,mouse...2,mouse...3,mouse...4
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
clones_percent_skin_unique,94.1176471,78.3582090,85.512367,80.753138
clones_percent_overlap_with_LN_only,0.3267974,0.3731343,0.000000,1.673640
clones_percent_overlap_with_spleen_only,3.9215686,14.5522388,11.660777,15.481172
clones_percent_overlap_with_spleen_and_LN,1.6339869,6.7164179,2.826855,2.092050
sequences_percent_skin_unique,89.8916968,63.2443532,72.709163,73.543689
sequences_percent_overlap_with_LN_only,0.1805054,0.2053388,0.000000,2.669903


## Mutated sequences

In [9]:
# results_all_samples_simple
# results_all_samples_with_SHM

results <- tibble(.rows = 8)
sequences_results <- tibble(.rows = 4)

for (Mouse in seq(1:4)) {
    
    mouse <- as.character(Mouse)
    skin_mouse <- paste0('E',  mouse )
    spleen_mouse <- paste0('S',  mouse )
    LN_mouse <- paste0('L',  mouse )
    
    new_names <- c('skin', 'spleen', 'LN')
    names(new_names) <- c(skin_mouse, spleen_mouse, LN_mouse)

    results_all_samples_SHM_skin <- results_all_samples_with_SHM |> 
        select(contains(mouse) | clonotype | v_mismatches) |> 
        rename(all_of(new_names)) |> 
        filter(skin > 0 & v_mismatches > 0)         

    total_unique_clones <- results_all_samples_SHM_skin |> pull(clonotype) |> n_distinct()
    total_unique_sequences <- results_all_samples_SHM_skin |> pull(skin) |> sum()


    clones_percent_skin_unique <- results_all_samples_SHM_skin |> filter(LN == 0 & spleen == 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_skin_unique <- results_all_samples_SHM_skin |> filter(LN == 0 & spleen == 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_LN_only <- results_all_samples_SHM_skin |> filter(LN  >  0 & spleen == 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_LN_only <- results_all_samples_SHM_skin |> filter(LN > 0 & spleen == 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_spleen_only <- results_all_samples_SHM_skin |> filter(LN ==  0 & spleen > 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_spleen_only <- results_all_samples_SHM_skin |> filter(LN  ==  0 & spleen > 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_spleen_and_LN <- results_all_samples_SHM_skin |> filter(LN >  0 & spleen > 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_spleen_and_LN <- results_all_samples_SHM_skin |> filter(LN  >  0 & spleen > 0) |> pull(skin) |> sum()*100/total_unique_sequences

    mouse_results <- c(clones_percent_skin_unique, clones_percent_overlap_with_LN_only, clones_percent_overlap_with_spleen_only, clones_percent_overlap_with_spleen_and_LN, sequences_percent_skin_unique, sequences_percent_overlap_with_LN_only, sequences_percent_overlap_with_spleen_only, sequences_percent_overlap_with_spleen_and_LN)


    results <- bind_cols(results, tibble(mouse = mouse_results))

}
row_names  <-  c('clones_percent_skin_unique', 'clones_percent_overlap_with_LN_only', 'clones_percent_overlap_with_spleen_only', 'clones_percent_overlap_with_spleen_and_LN', 'sequences_percent_skin_unique', 'sequences_percent_overlap_with_LN_only', 'sequences_percent_overlap_with_spleen_only', 'sequences_percent_overlap_with_spleen_and_LN')
results <- results |> mutate(Metric = row_names) |> relocate(Metric)
head(results)

write.csv(results, file = 'BCR_mutated_overlap_between_tissues.csv', row.names = FALSE)





New names:
• `mouse` -> `mouse...1`
• `mouse` -> `mouse...2`
New names:
• `mouse` -> `mouse...3`
New names:
• `mouse` -> `mouse...4`


Metric,mouse...1,mouse...2,mouse...3,mouse...4
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
clones_percent_skin_unique,92.576419,76.7241379,84.3049327,77.1573604
clones_percent_overlap_with_LN_only,0.000000,0.4310345,0.4484305,0.5076142
clones_percent_overlap_with_spleen_only,7.423581,18.1034483,13.4529148,19.7969543
clones_percent_overlap_with_spleen_and_LN,0.000000,4.7413793,1.7937220,2.5380711
sequences_percent_skin_unique,92.140921,64.9425287,73.9002933,73.7931034
sequences_percent_overlap_with_LN_only,0.000000,0.8620690,0.5865103,0.3448276


# D50

In [20]:
head(results_all_samples_simple)
head(results_all_samples_with_SHM)

,clonotype,E1,E2,E3,E4,L1,L2,L3,L4,S1,S2,S3,S4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CARAPYGNYRFAYW_IGHV9-3,5,0,0,0,0,0,0,0,2,0,0,0
2,CARYRSGYDAMDYW_IGHV1-18,4,0,0,0,0,0,0,0,0,0,0,0
3,CSIYYSFPFEYW_IGHV14-3,8,0,0,0,0,0,0,0,21,0,0,0
4,CARNGYYYAMDYW_IGHV2-2,5,0,0,0,0,0,2,5,7,7,14,2
5,CVRDGDYDVDWFAYW_IGHV10-3,5,0,0,0,0,0,0,0,0,0,0,0
6,CAKRYYGNVGYFDYW_IGHV1-64,4,0,0,0,0,0,0,0,2,0,0,0


,clonotype,v_mismatches,E1,E2,E3,E4,L1,L2,L3,L4,S1,S2,S3,S4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CARAPYGNYRFAYW_IGHV9-3_0,0,3,0,0,0,0,0,0,0,1,0,0,0
2,CARYRSGYDAMDYW_IGHV1-18_0,0,2,0,0,0,0,0,0,0,0,0,0,0
3,CSIYYSFPFEYW_IGHV14-3_11,11,3,0,0,0,0,0,0,0,5,0,0,0
4,CARNGYYYAMDYW_IGHV2-2_5,5,2,0,0,0,0,0,0,0,2,0,3,0
5,CVRDGDYDVDWFAYW_IGHV10-3_4,4,1,0,0,0,0,0,0,0,0,0,0,0
6,CAKRYYGNVGYFDYW_IGHV1-64_1,1,2,0,0,0,0,0,0,0,1,0,0,0


## Without considering SHM

In [18]:
tissues <- c('E', 'L', 'S')

#Initiating results data frame
results <- as.data.frame(matrix(nrow = 0,ncol = length(tissues)))
colnames(results)
rnames <- c()

#Calculate D50
for (mouse in seq(1:4)) {
    
    mouse <- as.character(mouse)
    result <- c()
    names <- c()

    for (tissue in tissues) {
        
        sample <- paste0(tissue, mouse)
        #Extracting data for cell type and mouse
        tissue_HTO_data <- results_all_samples_simple |>
                                    select(!!as.name(sample)|clonotype) |> 
                                    filter(!!as.name(sample) > 0) |> 
                                    arrange(desc(!!as.name(sample))) |>
                                    mutate(cumulative = cumsum(!!as.name(sample))) 

        total_sequences <- tissue_HTO_data |> summarize(sum(!!as.name(sample)))  |> as.numeric()
        print(total_sequences)

        #Calculating D50
        if (total_sequences < 20) {
            D50 <- NA 
        } else {
            L50 <- floor(total_sequences/2)
            number_unique_50 <- tissue_HTO_data |> filter(cumulative <= L50) %>% summarise(n_distinct(clonotype)) %>% as.numeric()
            number_unique_total <- tissue_HTO_data %>% summarise(n_distinct(clonotype)) %>% as.numeric()
            D50 <- number_unique_50*100/number_unique_total
        }
        result <- c(result, D50)
    }
    results <- rbind(results, result)
    rnames <- c(rnames, mouse)
    print(mouse)

}

print(rnames)
nrow(results)
colnames(results) <- paste0(tissues, '_D50')
colnames(results)

results <- results %>% mutate(Mouse=rnames) %>% arrange(Mouse) |> relocate(Mouse)
results

write.csv(results, file=paste0('D50_per_tissue.csv'), row.names=FALSE)

[1] "V1" "V2" "V3"

[1] 554
[1] 33612
[1] 35378
[1] "1"
[1] 487
[1] 143766
[1] 567435
[1] "2"
[1] 502
[1] 64265
[1] 273785
[1] "3"
[1] 412
[1] 115429
[1] 375287
[1] "4"
[1] "1" "2" "3" "4"


[1] 4

[1] "E_D50" "L_D50" "S_D50"

Mouse,E_D50,L_D50,S_D50
<chr>,<dbl>,<dbl>,<dbl>
1,17.97386,35.95274,25.31802
2,18.65672,37.39723,32.18361
3,18.37456,36.47094,36.81346
4,20.50209,38.69237,36.03583


## Considering SHM

In [22]:
tissues <- c('E', 'L', 'S')

#Initiating results data frame
results <- as.data.frame(matrix(nrow = 0,ncol = length(tissues)))
colnames(results)
rnames <- c()

#Calculate D50
for (mouse in seq(1:4)) {
    
    mouse <- as.character(mouse)
    result <- c()
    names <- c()

    for (tissue in tissues) {
        
        sample <- paste0(tissue, mouse)
        #Extracting data for cell type and mouse
        tissue_HTO_data <- results_all_samples_with_SHM |>
                                    select(!!as.name(sample)|clonotype) |> 
                                    filter(!!as.name(sample) > 0) |> 
                                    arrange(desc(!!as.name(sample))) |>
                                    mutate(cumulative = cumsum(!!as.name(sample))) 

        total_sequences <- tissue_HTO_data |> summarize(sum(!!as.name(sample)))  |> as.numeric()
        print(total_sequences)

        #Calculating D50
        if (total_sequences < 20) {
            D50 <- NA 
        } else {
            L50 <- floor(total_sequences/2)
            number_unique_50 <- tissue_HTO_data |> filter(cumulative <= L50) %>% summarise(n_distinct(clonotype)) %>% as.numeric()
            number_unique_total <- tissue_HTO_data %>% summarise(n_distinct(clonotype)) %>% as.numeric()
            D50 <- number_unique_50*100/number_unique_total
        }
        result <- c(result, D50)
    }
    results <- rbind(results, result)
    rnames <- c(rnames, mouse)
    print(mouse)

}

print(rnames)
nrow(results)
colnames(results) <- paste0(tissues, '_D50')
colnames(results)

results <- results %>% mutate(Mouse=rnames) %>% arrange(Mouse) |> relocate(Mouse)
results

write.csv(results, file=paste0('D50_per_tissue_with_SHM.csv'), row.names=FALSE)

[1] "V1" "V2" "V3"

[1] 554
[1] 33612
[1] 35378
[1] "1"
[1] 487
[1] 143766
[1] 567435
[1] "2"
[1] 502
[1] 64265
[1] 273785
[1] "3"
[1] 412
[1] 115429
[1] 375287
[1] "4"
[1] "1" "2" "3" "4"


[1] 4

[1] "E_D50" "L_D50" "S_D50"

Mouse,E_D50,L_D50,S_D50
<chr>,<dbl>,<dbl>,<dbl>
1,30.75000,45.96489,41.60504
2,33.51499,45.18522,39.50227
3,32.52688,46.24881,43.25326
4,32.67974,45.90149,42.45867


# Remaking file to incorporate SHM and replicates and isotype

In [11]:
samples <- c('E1', 'E2', 'E3', 'E4', 'S1', 'S2', 'S3', 'S4')

files <- c('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E1__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E1_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E2__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E2_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E3__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E3_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E4__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/E4_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L1__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L1_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L2__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L2_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L3__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L3_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L4__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/L4_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S1__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S1_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S2__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S2_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S3__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S3_r__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S4__IGH.csv' ,
'./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/S4_r__IGH.csv')

In [7]:
sample1 <- paste0('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/', 'E1', '__IGH.csv' )
sample2 <- paste0('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/', 'E1', '_r__IGH.csv' )

table1 <- read.csv(file = sample1)
table2 <- read.csv(file = sample2)

table1 <- table1 |>
select(all_of(c('junction_aa', 'v_call', 'v_mismatches', 'c_call')))

table2 <- table2 |>
select(all_of(c('junction_aa', 'v_call', 'v_mismatches', 'c_call')))

table <- bind_rows(table1, table2)  |>
    rename('junction_aa' = 'CDR3') |>
    mutate(clonotype  =  paste(CDR3, v_call,sep = '_')) |> 
    select(-c(CDR3, v_call)) |>
    relocate(clonotype) |> 
    add_count(clonotype, name = 'clonotype_count')  |>
    distinct()

nrow(table)    
table |> pull(clonotype) |> n_distinct()
head(table)


[1] 403

[1] 306

,clonotype,v_mismatches,c_call,clonotype_count
,<chr>,<int>,<chr>,<int>
1,CARAPYGNYRFAYW_IGHV9-3,0,IGHA,5
2,CARYRSGYDAMDYW_IGHV1-18,0,IGHA,4
3,CSIYYSFPFEYW_IGHV14-3,11,IGHA,8
4,CARNGYYYAMDYW_IGHV2-2,5,IGHA,5
5,CVRDGDYDVDWFAYW_IGHV10-3,4,IGHA,5
6,CAKRYYGNVGYFDYW_IGHV1-64,1,IGHA,4


In [12]:
results_all_samples_simple <- tibble()
results_all_samples_with_SHM <- tibble()

for (sample in samples) {
    sample1 <- paste0('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/', sample, '__IGH.csv' )
    sample2 <- paste0('./2404-iR-0005_NIH_additional_analysisfinal/cdr2_fr4_airr/', sample, '_r__IGH.csv' )

    table1 <- read.csv(file = sample1)
    table2 <- read.csv(file = sample2)

    table1 <- table1 |>
    select(all_of(c('junction_aa', 'v_call', 'v_mismatches', 'c_call')))

    table2 <- table2 |>
    select(all_of(c('junction_aa', 'v_call', 'v_mismatches',  'c_call')))

    table <- bind_rows(table1, table2)  |>
        rename('junction_aa' = 'CDR3') |>
        mutate(clonotype  =  paste(CDR3, v_call, v_mismatches, sep = '_')) |> 
        select(-c(CDR3, v_call)) |>
        relocate(clonotype) 
        
    
    table_simple  <- bind_rows(table1, table2)  |>
        rename('junction_aa' = 'CDR3') |>
        select(-v_mismatches) |> 
        mutate(clonotype  =  paste(CDR3, v_call,sep = '_')) |> 
        select(-c(CDR3, v_call)) |>
        relocate(clonotype) 
        
    
    if (str_detect(string = sample,pattern = 'S')) {
        table <- table |>
            filter(c_call == 'IGHM') |>
            select(-c_call) |>
            add_count(clonotype, v_mismatches, name = sample)  |>
            distinct()

        table_simple <- table_simple |>
            filter(c_call == 'IGHM') |>
            select(-c_call) |>
            add_count(clonotype, name =  sample)   |>
            distinct()
    } else {
        table <- table |>
            select(-c_call) |>
            add_count(clonotype, v_mismatches, name = sample)  |>
            distinct()

        table_simple <- table_simple |>
            select(-c_call) |>
            add_count(clonotype, name =  sample)   |>
            distinct()
    }

    if (nrow(results_all_samples_simple) == 0) {
        results_all_samples_simple <- table_simple
        results_all_samples_with_SHM <- table 
    } else {
        results_all_samples_simple <- full_join(results_all_samples_simple, table_simple, by = 'clonotype') |>
                mutate_all(~replace(., is.na(.), 0))  

        results_all_samples_with_SHM <- full_join(results_all_samples_with_SHM, table, by = c( 'clonotype' ,  'v_mismatches' )) |>
                mutate_all(~replace(., is.na(.), 0))
    }
        
}

head(results_all_samples_simple)
head(results_all_samples_with_SHM)

write.csv(results_all_samples_simple, file = 'results_all_samples_simple_spleen_IGHM_only.csv', row.names = F)

write.csv(results_all_samples_with_SHM, file = 'results_all_samples_with_SHM_spleen_IGHM_only.csv', row.names = F)

,clonotype,E1,E2,E3,E4,S1,S2,S3,S4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CARAPYGNYRFAYW_IGHV9-3,5,0,0,0,0,0,0,0
2,CARYRSGYDAMDYW_IGHV1-18,4,0,0,0,0,0,0,0
3,CSIYYSFPFEYW_IGHV14-3,8,0,0,0,0,0,0,0
4,CARNGYYYAMDYW_IGHV2-2,5,0,0,0,0,2,5,2
5,CVRDGDYDVDWFAYW_IGHV10-3,5,0,0,0,0,0,0,0
6,CAKRYYGNVGYFDYW_IGHV1-64,4,0,0,0,0,0,0,0


,clonotype,v_mismatches,E1,E2,E3,E4,S1,S2,S3,S4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CARAPYGNYRFAYW_IGHV9-3_0,0,3,0,0,0,0,0,0,0
2,CARYRSGYDAMDYW_IGHV1-18_0,0,2,0,0,0,0,0,0,0
3,CSIYYSFPFEYW_IGHV14-3_11,11,3,0,0,0,0,0,0,0
4,CARNGYYYAMDYW_IGHV2-2_5,5,2,0,0,0,0,0,0,0
5,CVRDGDYDVDWFAYW_IGHV10-3_4,4,1,0,0,0,0,0,0,0
6,CAKRYYGNVGYFDYW_IGHV1-64_1,1,2,0,0,0,0,0,0,0


In [26]:
head(results_all_samples_simple |> arrange(desc(E1)))

,clonotype,E1,E2,E3,E4,S1,S2,S3,S4
,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,CARNYYAMDYW_IGHV2-2,27,0,0,0,0,3,3,4
2,CARFTTDYYAMDYW_IGHV1-19,19,0,0,0,0,0,0,0
3,CVRQSYYSKGGYWYFDVW_IGHV10-1,14,0,0,0,0,0,0,0
4,CARSYDYDGWYAMDYW_IGHV8-12,13,0,0,0,0,0,0,0
5,CARSYYYGSSYFDYW_IGHV1-80,12,0,0,0,1,1,0,0
6,CVSYYGNYEYYAMDYW_IGHV9-1,10,0,0,0,0,0,0,0


# Percentage overlap with spleen IGHM 

## Total sequences

In [27]:
# results_all_samples_simple
# results_all_samples_with_SHM

results <- tibble(.rows = 4)
#sequences_results <- tibble(.rows = 4)

for (Mouse in seq(1:4)) {
    
    mouse <- as.character(Mouse)
    skin_mouse <- paste0('E',  mouse )
    spleen_mouse <- paste0('S',  mouse )
    
    new_names <- c('skin', 'spleen')
    names(new_names) <- c(skin_mouse, spleen_mouse)

    results_all_samples_simple_skin <- results_all_samples_simple |> 
        select(contains(mouse) | clonotype) |> 
        rename(all_of(new_names)) |> 
        filter(skin > 0)         
    total_unique_clones <- results_all_samples_simple_skin |> pull(clonotype) |> n_distinct()
    total_unique_sequences <- results_all_samples_simple_skin |> pull(skin) |> sum()


    clones_percent_skin_unique <- results_all_samples_simple_skin |> filter(spleen == 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_skin_unique <- results_all_samples_simple_skin |> filter(spleen == 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_spleen_IGHM <- results_all_samples_simple_skin |> filter(spleen > 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_spleen_IGHM <- results_all_samples_simple_skin |> filter(spleen > 0) |> pull(skin) |> sum()*100/total_unique_sequences

    mouse_results <- c(clones_percent_skin_unique, clones_percent_overlap_with_spleen_IGHM, sequences_percent_skin_unique, sequences_percent_overlap_with_spleen_IGHM)


    results <- bind_cols(results, tibble(mouse = mouse_results))

}
row_names  <-  c('clones_percent_skin_unique', 'clones_percent_overlap_with_spleen_IGHM', 'sequences_percent_skin_unique', 'sequences_percent_overlap_with_spleen_IGHM')
results <- results |> mutate(Metric = row_names) |> relocate(Metric)
head(results)

write.csv(results, file = 'BCR_total_overlap_between_skin_and_spleen_IGHM.csv', row.names = FALSE)





New names:
• `mouse` -> `mouse...1`
• `mouse` -> `mouse...2`
New names:
• `mouse` -> `mouse...3`
New names:
• `mouse` -> `mouse...4`


Metric,mouse...1,mouse...2,mouse...3,mouse...4
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
clones_percent_skin_unique,97.385621,87.31343,93.286219,92.887029
clones_percent_overlap_with_spleen_IGHM,2.614379,12.68657,6.713781,7.112971
sequences_percent_skin_unique,95.848375,80.08214,88.047809,88.349515
sequences_percent_overlap_with_spleen_IGHM,4.151625,19.91786,11.952191,11.650485


## Mutated sequences

In [13]:
# results_all_samples_simple
# results_all_samples_with_SHM

results <- tibble(.rows = 4)
#sequences_results <- tibble(.rows = 4)

for (Mouse in seq(1:4)) {
    
    mouse <- as.character(Mouse)
    skin_mouse <- paste0('E',  mouse )
    spleen_mouse <- paste0('S',  mouse )
    
    new_names <- c('skin', 'spleen')
    names(new_names) <- c(skin_mouse, spleen_mouse)

    results_all_samples_SHM_skin <- results_all_samples_with_SHM |> 
        select(contains(mouse) | clonotype | v_mismatches) |> 
        rename(all_of(new_names)) |> 
        filter(skin > 0 & v_mismatches > 0)         
    total_unique_clones <- results_all_samples_SHM_skin |> pull(clonotype) |> n_distinct()
    total_unique_sequences <- results_all_samples_SHM_skin |> pull(skin) |> sum()


    clones_percent_skin_unique <- results_all_samples_SHM_skin |> filter(spleen == 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_skin_unique <- results_all_samples_SHM_skin |> filter(spleen == 0) |> pull(skin) |> sum()*100/total_unique_sequences

    clones_percent_overlap_with_spleen_IGHM <- results_all_samples_SHM_skin |> filter(spleen > 0) |> pull(clonotype) |> n_distinct()*100/total_unique_clones
    sequences_percent_overlap_with_spleen_IGHM <- results_all_samples_SHM_skin |> filter(spleen > 0) |> pull(skin) |> sum()*100/total_unique_sequences

    mouse_results <- c(clones_percent_skin_unique, clones_percent_overlap_with_spleen_IGHM, sequences_percent_skin_unique, sequences_percent_overlap_with_spleen_IGHM)


    results <- bind_cols(results, tibble(mouse = mouse_results))

}
row_names  <-  c('clones_percent_skin_unique', 'clones_percent_overlap_with_spleen_IGHM', 'sequences_percent_skin_unique', 'sequences_percent_overlap_with_spleen_IGHM')
results <- results |> mutate(Metric = row_names) |> relocate(Metric)
head(results)

write.csv(results, file = 'BCR_mutated_overlap_between_skin_and_spleen_IGHM.csv', row.names = FALSE)


New names:
• `mouse` -> `mouse...1`
• `mouse` -> `mouse...2`
New names:
• `mouse` -> `mouse...3`
New names:
• `mouse` -> `mouse...4`


Metric,mouse...1,mouse...2,mouse...3,mouse...4
<chr>,<dbl>,<dbl>,<dbl>,<dbl>
clones_percent_skin_unique,99.1266376,91.810345,98.654709,97.461929
clones_percent_overlap_with_spleen_IGHM,0.8733624,8.189655,1.345291,2.538071
sequences_percent_skin_unique,99.1869919,88.793103,97.360704,97.586207
sequences_percent_overlap_with_spleen_IGHM,0.8130081,11.206897,2.639296,2.413793
